# PINNS-based solution for 2D stabilized Darcy equations

## Overview

Darcy equation is a second-order, elliptic PDE (partial differential equation), which describes the flow through a porous medium at low speed. It is widely used in hydraulic engineering and petroleum engineering. The Darcy equation was originally formulated by Henry Darcy on the basis of experimental results of permeability experiments in sandy soil, and later derived from the Navier-Stokes equation by Stephen Whitaker via the homogenization method.

It is difficult to obtain a generalized analytical solution of the Darcy equation for the permeability field of different fluids and the numerical method is usually used to solve the Darcy governing equation describing a specific scenario, and then the pressure field and velocity field of flow under the scenario are simulated. The numerical simulation results of Darcy flow can be used for further scientific research and engineering practice. Finite element method (FEM) for Darcy equation is designed to work with finite element spaces. In addition, for many problems of practical interest, some physical terms of the Darcy equation will be stabilized. The finite element method is designed in the standard finite element grid space. The higher accuracy required for numerical solution, the more fine the grid needs to be divided, and it costs larger time and storage overhead.

As the research of parallel algorithm for numerical solution tends to slow down, the method based on neural network has been developed and achieved the solution accuracy close to the traditional numerical method. In 2019, the Applied Mathematics team of Brown University proposed a Physics-informed Neural Networks (PINNs) and provided a complete code framework to construct PINNs for solving a wide variety of PDEs. In this case, MindFlow suite of fluid equations is used to solve the two-dimensional stabilized Darcy equation based on PINNs method.

## Problem Description

Considering the two-dimensional cube $\Omega=(0, 1)\times(0, 1)$, The boundary of the cube is $\Gamma$. Ignoring the effects of gravity, in the range of $\Omega$, the two-dimensional stabilized Darcy equation satisfied by the fluid pressure $p$ and velocity $u$ is as follows:

$$
\begin{align}
u + \nabla p &= 0, (x, y)\in\Omega\\
\nabla \cdot u &= f, (x, y)\in\Omega
\end{align}
$$

The Dirichlet boundary conditions are used in this case in the following form:

$$
\begin{align}
u_x &= -2 \pi cos(2 \pi x) cos(2 \pi y) &(x, y)\in\Gamma\\
u_y &= 2 \pi sin(2 \pi x) sin(2 \pi y) &(x, y)\in\Gamma\\
p &= sin(2 \pi x) cos(2 \pi y) &(x, y)\in\Gamma
\end{align}
$$

In which $f$ is **forcing function** in the Darcy equation. In this case, **forcing function** $f$ is used to learn the mapping $(x, y) \mapsto (u, p)$ from position to corresponding physical quantities when **forcing function** $f$ is $8 \pi^2 sin(2 \pi x)cos(2 \pi y)$. So that the solution of Darcy equation is realized.

In [1]:
import numpy as np

from mindspore.common import set_seed
from mindspore import nn
from mindspore.train import DynamicLossScaleManager
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

In [2]:
from mindflow.loss import Constraints
from mindflow.solver import Solver
from mindflow.common import L2
from mindflow.utils import load_yaml_config
from mindflow.common import LossAndTimeMonitor
from mindflow.cell import FCSequential

set_seed(123456)
np.random.seed(123456)

In [3]:
config = load_yaml_config("darcy_cfg.yaml")

### Training Dataset Construction

For the training dataset, this case conducts random sampling according to the problem domain and boundary conditions. The sampling configuration information is as follows, and samples are collected according to uniform distribution. The problem domain of cube is constructed, and then the known problem domain and boundary are sampled.


In [4]:
from scipy.constants import pi as PI

from mindflow.data import Dataset
from mindflow.geometry import Rectangle
from mindflow.geometry import generate_sampling_config

def create_random_dataset(config, name):
    """create training dataset by online sampling"""
    # define geometry
    coord_min = config["geometry"]["coord_min"]
    coord_max = config["geometry"]["coord_max"]
    data_config = config["data"]

    flow_region = Rectangle(
        name,
        coord_min=coord_min,
        coord_max=coord_max,
        sampling_config=generate_sampling_config(data_config),
    )
    geom_dict = {flow_region: ["domain", "BC"]}

    # create dataset for train
    dataset = Dataset(geom_dict)
    return dataset

In [5]:
geom_name = "flow_region"
# create train dataset
flow_train_dataset = create_random_dataset(config, geom_name)
train_data = flow_train_dataset.create_dataset(
    batch_size=config["train_batch_size"], shuffle=True, drop_remainder=True
)

## Model Construction

This example uses a simple fully-connected network with a depth of 6 layers and the activation function is the tanh function.

In [7]:
# network model
model = FCSequential(
    in_channels=config["model"]["input_size"],
    out_channels=config["model"]["output_size"],
    neurons=config["model"]["neurons"],
    layers=config["model"]["layers"],
    residual=config["model"]["residual"],
    act=config["model"]["activation"],
    weight_init=config["model"]["weight_init"]
)

## Problem

In [9]:
from scipy.constants import pi as PI
from mindspore import ms_function
from mindspore import ops
from mindspore import Tensor
import mindspore.common.dtype as mstype

from mindflow.operators import Grad
from mindflow.pde import Problem

class Darcy2D(Problem):
    r"""
    The steady-state 2D Darcy flow's equations with Dirichlet boundary condition

    Args:
      model (Cell): The solving network.
      domain_name (str): The corresponding column name of data which governed by maxwell's equation.
      bc_name (str): The corresponding column name of data which governed by boundary condition.
    """

    def __init__(self, model, domain_name=None, bc_name=None):
        super(Darcy2D, self).__init__()
        self.domain_name = domain_name
        self.bc_name = bc_name
        self.model = model
        self.grad = Grad(self.model)
        self.sin = ops.Sin()
        self.cos = ops.Cos()

        # constants
        self.pi = Tensor(PI, mstype.float32)

    def force_function(self, in_x, in_y):
        """"forcing function in Darcy Equation"""
        return 8 * self.pi**2 * self.sin(2 * self.pi * in_x) * self.cos(2 * self.pi * in_y)

    @ms_function
    def governing_equation(self, *output, **kwargs):
        """darcy equation"""
        u_x, u_y, _ = ops.split(output[0], axis=1, output_num=3)

        data = kwargs[self.domain_name]
        in_x = ops.Reshape()(data[:, 0], (-1, 1))
        in_y = ops.Reshape()(data[:, 1], (-1, 1))

        duxdx = ops.Cast()(self.grad(data, 0, 0, output[0]), mstype.float32)
        duydy = ops.Cast()(self.grad(data, 1, 1, output[0]), mstype.float32)
        dpdx = ops.Cast()(self.grad(data, 0, 2, output[0]), mstype.float32)
        dpdy = ops.Cast()(self.grad(data, 1, 2, output[0]), mstype.float32)

        loss_1 = -1 * (duxdx + duydy - self.force_function(in_x, in_y))
        loss_2 = 1 * (u_x + dpdx)
        loss_3 = 2 * self.pi * (u_y + dpdy)

        return ops.Concat(1)((loss_1, loss_2, loss_3))

    @ms_function
    def boundary_condition(self, *output, **kwargs):
        """Dirichlet boundary condition"""

        out_vars = output[0]
        u_x, u_y, pressure = ops.split(out_vars, axis=1, output_num=3)
        data = kwargs[self.bc_name]
        in_x = ops.Reshape()(data[:, 0], (-1, 1))
        in_y = ops.Reshape()(data[:, 1], (-1, 1))
        ux_boundary = -1 * (
            u_x - (-2 * self.pi * self.cos(2 * self.pi * in_x) * self.cos(2 * self.pi * in_y))
        )

        uy_boundary = 1 * (
            u_y - (2 * self.pi * self.sin(2 * self.pi * in_x) * self.sin(2 * self.pi * in_y))
        )

        p_boundary = (
            2 * self.pi * (pressure - self.sin(2 * self.pi * in_x) * self.cos(2 * self.pi * in_y))
        )
        return ops.Concat(1)((ux_boundary, uy_boundary, p_boundary))

## Constraints

In [10]:
# define problem and Constraints
darcy_problem = [
    Darcy2D(model=model) for _ in range(flow_train_dataset.num_dataset)
]
train_constraints = Constraints(flow_train_dataset, darcy_problem)

## optimizer

In [11]:
# optimizer
params = model.trainable_params()
optim = nn.Adam(params, learning_rate=config["optimizer"]["lr"])

## Model Training


In [12]:
# solver
solver = Solver(
    model,
    optimizer=optim,
    mode="PINNs",
    train_constraints=train_constraints,
    test_constraints=None,
    metrics={"l2": L2(), "distance": nn.MAE()},
    loss_scale_manager=DynamicLossScaleManager(),
)

In [13]:
# define callbacks
callbacks = [LossAndTimeMonitor(len(flow_train_dataset))]

if config["save_ckpt"]:
    ckpt_config = CheckpointConfig(save_checkpoint_steps=10, keep_checkpoint_max=2)
    ckpoint_cb = ModelCheckpoint(
        prefix="ckpt_darcy", directory=config["save_ckpt_path"], config=ckpt_config
    )
    callbacks += [ckpoint_cb]

In [ ]:
solver.train(
    epoch=config["train_epoch"], train_dataset=train_data, callbacks=callbacks
)

## Model Evaluation and Visualizetion

After training, all data points in the flow field can be inferred. And related results can be visualized.

In [ ]:
from src import visual_result
visual_result(model, config)

![result](images/result.png)